# Call Common Functions

In [ ]:
%run /utils/common_functions



In [ ]:
from delta.tables import *
from notebookutils import mssparkutils
spark.conf.set("spark.databricks.delta.autoCompact.enabled","true")
spark.sql("SET spark.databricks.delta.schema.autoMerge.enabled = true")
spark.conf.set("spark.databricks.io.cache.enabled", "true")
spark.conf.set("spark.sql.sources.partitionOverwriteMode","dynamic")
spark.conf.set("spark.sql.legacy.timeParserPolicy","LEGACY")
spark.conf.set("spark.sql.adaptive.enabled","true")
spark.conf.set("spark.sql.adaptive.coalescePartitions.enabled","true")
spark.conf.set("spark.sql.adaptive.skewJoin.enabled","true")
spark.conf.set("spark.databricks.adaptive.autoOptimizeShuffle.enabled","true")

In [ ]:
%%sql
create database if not exists structured;

# Copying all common functions

In [ ]:
 def file_exists(adls_path):
      try:
        if len(mssparkutils.fs.ls(adls_path)) > 0:
          print("file Exists::", adls_path)
          return True
      except Exception as e:
        if 'java.io.FileNotFoundException' in str(e):
          return False
        else:
          raise e    

def get_target_table_ref(target_path):
    try:
      #sql = f"CREATE TABLE IF NOT EXISTS {db}.{target_table} USING DELTA LOCATION '{target_path}'"
      print("get_target_table_ref i/p::",target_path)
      #spark.sql(sql)
      delta_table = DeltaTable.forPath(spark, target_path)  
      return delta_table
    except Exception as e:
      raise e

def execute_merge(target_table, staged_updates, where_condition): ## for ctl_config_varaibles: where_condition :
      try:
        target_table.alias("target").merge(staged_updates.alias("source"),where_condition)\
               .whenMatchedUpdateAll()\
              .whenNotMatchedInsertAll()\
              .execute()
        print("table upserted")      
        return target_table.toDF()
      except Exception as e:
         raise e                

# Main( )

In [ ]:

def perform_merge(dict_tables):
  
    try: 
      
      
        
      for keys,values in dict_tables.items():
        
        print("#################################perform_merge########")
        print("table::", keys)
        print("table_path::",dict_tables[keys]["path"])
        #dict_tables[keys]["path"] = structured_adls_path + keys
        #source_df = spark.read.load(raw_adls_path+dict_tables[keys]["source_path"],format="delta")
        source_df = dict_tables[keys]["source_df"]
        
        print("target path::", dict_tables[keys]["path"])
        if file_exists(dict_tables[keys]["path"]):
          print(dict_tables[keys],"::table_exists")
          execute_merge(get_target_table_ref(dict_tables[keys]["path"]),source_df,dict_tables[keys]["where_condition"])
        elif dict_tables[keys]["partition"]:
          print(keys," table does not exist and is partitioned... going to create it...partion key::",dict_tables[keys]["partition"])
          source_df.write.mode("overwrite").format("delta").option("path", dict_tables[keys]["path"])\
          .partitionBy(dict_tables[keys]["partition"])\
          .saveAsTable(dict_tables[keys]["target_table"])
          print(keys,"::table_created" )
        else:
          print(keys," table does not exist, going to create it...")
          source_df.write.mode("overwrite").format("delta").option("path", dict_tables[keys]["path"])\
          .saveAsTable(dict_tables[keys]["target_table"])
          print(keys,"::table_created" )
       
 
    except Exception as e:
      raise e  
